In [3]:
import torch
import torch.nn as nn
import torch.fft as fft

In [ ]:
'''' 
Using Nishizaki et al. (2019)'s suggestions for ResNet2

'''' 

In [ ]:
class DBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MyBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=2, padding =1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLu(inplace = True)
        self.conv2 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernal_size = 3, stride = 1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        output1 = self.conv1(x)

        output2 = self.bn1(x)
        output2 = self.relu(output2)
        output2 = self.conv1(output2)
        output2 = self.bn2(output2)
        output2 = self.relu(output2)
        output2 = self.conv2(output2)

        output = output1 + output2
        return output

In [6]:
class UBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MyBlock, self).__init__()
        self.tconv1 = nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=2, padding = 1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLu(inplace = True)
        self.conv2 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride = 1, padding = 1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
    def forward(self, x):
        output1 = self.tconv1(x)
        
        output2 = self.bn1(x)
        output2 = self.relu(output2)
        output2 = self.tconv(output2)
        output2 = self.bn2(output2)
        output2 = self.relu(output2)
        output2 = self.conv2(output2)

        output = output1 + output2
        return output


In [7]:
class SBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MyBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=1, padding=1)
    def forward(self, x):
        output = self.conv1(x)

In [8]:
class ResNet2(nn.Module):
    def __init__(self, in_channels = 1, out_channels = 64):
        super(ResNet2, self).__init__()
        self.s1 = SBlock(in_channels=in_channels, out_channels=out_channels)
        self.s2 = SBlock(in_channels=out_channels, out_channels=out_channels)
        self.d1 = DBlock(in_channels=in_chanels,out_channels=out_channels)
        self.d2 = DBlock(in_channels=out_channels, out_channels=out_channels)
        self.u1 = UBlock(in_channels=out_channels, out_channels=out_channels)
        self.u2 = UBlock(in_channels=out_channels, out_channels=out_channels)
    def forward(self, x):
        output_s1 = self.s1(x)

        output_inside = self.d1(x)
        
        output_s2 = self.s2(output_inside)

        output_du = self.d2(output_inside)
        output_du = self.u1(output_du)
        output_du = output_du + output_s2
        output_du = self.u2(output_du)

        output = output_s1 + output_du
        return output



In [ ]:
'''' 
Training ResNet2

    Requires training set.
''''

In [10]:
import numpy as np
import matplotlib.pyplot as plt